In [1]:
from typing import Dict, Union
import sys
import os 
from pathlib import Path
import requests
import socket
import pandas as pd
import webbrowser
from requests_oauthlib import OAuth2Session
import gi
import tempfile
gi.require_version('NM', '1.0')
from gi.repository import GLib, NM

In [2]:
sys.path.append(str(Path().absolute().parent))

In [3]:
from eduvpn.oauth2 import one_request, get_open_port
from eduvpn.crypto import gen_code_challenge, gen_code_verifier, common_name_from_cert

# Settings

In [10]:
# client settings
DISCO_URI = 'https://disco.eduvpn.org/'
ORGANISATION_URI = DISCO_URI + "organization_list.json"
SERVER_URI = DISCO_URI + "server_list.json"
client_id = "org.eduvpn.app.linux"
scope = ["config"]
code_challenge_method = "S256"

In [5]:
# our configuration
organisation = 'SURFnet bv'
server = 'Demo'
institute = 'Demo'
LANGUAGE = 'nl'
COUNTRY = "nl-NL"

# Utils

In [6]:
def extract_translation(d: Union[str, Dict[str, str]]):
    if type(d) != dict:
        return d
    for m in [COUNTRY, LANGUAGE, 'en-US', 'en']:
        try:
            return d[m]
        except KeyError:
            continue
    return list(d.values())[0]  # otherwise just return first in list

# server list

In [15]:
servers_response = requests.get(SERVER_URI)
server_list = pd.DataFrame(servers_response.json()['server_list'])
server_list['display_name'] = server_list['display_name'].apply(extract_translation)
server_list

,server_type,base_url,display_name,public_key_list,support_contact
0,secure_internet,https://eduvpn.rash.al/,Albania,[Xv3l24gbMX8NtTnFQbWO2fGKPwKuc6EbjQDv8qw2GVk],[mailto:helpdesk@rash.al]
1,secure_internet,https://gdpt-eduvpndev1.tnd.aarnet.edu.au/,Australië,[HpY5RKF0OzYcYUcogKzgt1MvC6CxBmDJoUBsyiKjioA],NaN
2,secure_internet,https://eduvpn.deic.dk/,Denemarken,[bRTz33KIuYo_w_-AbzNtdmLDqIm11_eGiHXQniojxY4],[mailto:eduvpn@deic.dk]
3,secure_internet,https://eduvpn.eenet.ee/,Estonia,[jGpivOdwCRoLlexYKQjulZPPP4s3d9SVBFslI6RroAo],[mailto:eduvpn@eenet.ee]
4,secure_internet,https://eduvpn1.funet.fi/,Finland,[H4NTnM18BJgU_B3r8OBDVblBSfozB2Zu97I_ag2whmM],[mailto:eduvpn@csc.fi]
5,secure_internet,https://eduvpn-poc.renater.fr/,Frankrijk,[ePVNzE15h0yS6Xf3s8nJWmc8V6FeFziA3TZr0uOacFg],[https://assistance.renater.fr/]
6,secure_internet,https://eduvpn1.eduvpn.de/,Duitsland,[QjJHMit3vhHwLKi-fu2dXXSxMxnkskFVS3hMwyCnWQs],"[mailto:eduvpn@dfn.de, tel:+49308842999120]"
7,secure_internet,https://eduvpn.marwan.ma/,Marokko,[aX-El_yRPdcUDF5S2smQ-9U7BzB35_1RtFYSjbHfEz8],"[mailto:eduvpn-support@marwan.ma, tel:+2127000..."
8,secure_internet,https://guest.eduvpn.no/,Noorwegen,[qOLCcqXWZm9nmjsrwiJQxxWD606vDEJ2MIcc85oJmnE],[mailto:kontakt@uninett.no]
9,secure_internet,https://vpn.pern.edu.pk/,Pakistan,[LKWFZblpTFvFDY4E_0tnD8yHpK-iOSrJop_1x-A4cQ8],[mailto:eduvpn@pern.edu.pk]


# institute list

In [27]:
institute_list = server_list[server_list['server_type'] == 'institute_access'].drop(['server_type'], axis=1)
institute_list

,base_url,display_name,public_key_list,support_contact
14,https://sunset.nuonet.fr/,CNOUS,NaN,[mailto:support-technique-nuo@listes.nuonet.fr]
15,https://eduvpn-csc.funet.fi/,CSC - IT Center for Science Ltd.,NaN,[mailto:eduvpn@csc.fi]
16,https://demo.eduvpn.nl/,Demo,NaN,[mailto:eduvpn@surfnet.nl]
17,https://access.diak.fi/,DIAK,NaN,"[mailto:tuki@diak.fi, tel:+358294696070]"
18,https://differ.eduvpn.nl/,DIFFER,NaN,NaN
19,https://egi.eduvpn.nl/,EGI Foundation,NaN,NaN
20,https://eduvpn.ensma.fr/,Ensma,NaN,[mailto:dsi@ensma.fr]
21,https://eur.eduvpn.nl/,Erasmus University Rotterdam,NaN,NaN
22,https://prod-eduvpn01.geant.org/,GÉANT,NaN,[mailto:it@geant.org]
23,https://eduvpn.heanet.ie/,HEAnet,NaN,NaN


# organisation list

In [36]:
organisation_response = requests.get(ORGANISATION_URI)
organization_list = pd.DataFrame(organisation_response.json()['organization_list'])
organization_list['display_name'] = organization_list['display_name'].apply(extract_translation)
organization_list['keyword_list'] = organization_list['keyword_list'].apply(extract_translation)
organization_list

,display_name,org_id,secure_internet_home,keyword_list
0,Business Academy Aarhus,http://adfs.eaaa.dk/adfs/services/trust,https://eduvpn.deic.dk/,NaN
1,KMD DSPARE3,http://dans-support-04.dans-idp-dev01.northeur...,https://eduvpn.deic.dk/,NaN
2,KMD DSUP_PATCH1,https://dans-idp.kmd.dk:7080,https://eduvpn.deic.dk/,NaN
3,KMD DANSWRC2,http://dans-support-02.dans-dp-dev01.northeuro...,https://eduvpn.deic.dk/,NaN
4,IBA International Business Academy,http://sso.basyd.dk/adfs/services/trust,https://eduvpn.deic.dk/,NaN
...,...,...,...,...
572,Lentiz Onderwijsgroep,http://adfs.mijnlentiz.nl/adfs/services/trust,https://nl.eduvpn.org/,lentiz onderwijsgroep
573,Stichting Nuffic,https://sts.windows.net/0b883569-39c0-4403-a6a...,https://nl.eduvpn.org/,nuffic ep unesco
574,Hogeschool Utrecht,https://sts.hu.nl/adfs/services/trust,https://nl.eduvpn.org/,hogeschool hu hogeschoolutrecht hu university ...
575,Rijksinstituut voor Volksgezondheid en Milieu,http://fs.rivm.nl/adfs/services/trust,https://nl.eduvpn.org/,rivm volk gezondheid milieu rijk


# Secure internet

In [35]:
secure_internet_list = server_list[server_list['server_type'] == 'secure_internet'].drop(['server_type'], axis=1)
secure_internet_list

,base_url,display_name,public_key_list,support_contact
0,https://eduvpn.rash.al/,Albania,[Xv3l24gbMX8NtTnFQbWO2fGKPwKuc6EbjQDv8qw2GVk],[mailto:helpdesk@rash.al]
1,https://gdpt-eduvpndev1.tnd.aarnet.edu.au/,Australië,[HpY5RKF0OzYcYUcogKzgt1MvC6CxBmDJoUBsyiKjioA],NaN
2,https://eduvpn.deic.dk/,Denemarken,[bRTz33KIuYo_w_-AbzNtdmLDqIm11_eGiHXQniojxY4],[mailto:eduvpn@deic.dk]
3,https://eduvpn.eenet.ee/,Estonia,[jGpivOdwCRoLlexYKQjulZPPP4s3d9SVBFslI6RroAo],[mailto:eduvpn@eenet.ee]
4,https://eduvpn1.funet.fi/,Finland,[H4NTnM18BJgU_B3r8OBDVblBSfozB2Zu97I_ag2whmM],[mailto:eduvpn@csc.fi]
5,https://eduvpn-poc.renater.fr/,Frankrijk,[ePVNzE15h0yS6Xf3s8nJWmc8V6FeFziA3TZr0uOacFg],[https://assistance.renater.fr/]
6,https://eduvpn1.eduvpn.de/,Duitsland,[QjJHMit3vhHwLKi-fu2dXXSxMxnkskFVS3hMwyCnWQs],"[mailto:eduvpn@dfn.de, tel:+49308842999120]"
7,https://eduvpn.marwan.ma/,Marokko,[aX-El_yRPdcUDF5S2smQ-9U7BzB35_1RtFYSjbHfEz8],"[mailto:eduvpn-support@marwan.ma, tel:+2127000..."
8,https://guest.eduvpn.no/,Noorwegen,[qOLCcqXWZm9nmjsrwiJQxxWD606vDEJ2MIcc85oJmnE],[mailto:kontakt@uninett.no]
9,https://vpn.pern.edu.pk/,Pakistan,[LKWFZblpTFvFDY4E_0tnD8yHpK-iOSrJop_1x-A4cQ8],[mailto:eduvpn@pern.edu.pk]


# make selection

In [50]:
institute_info = institute_access_list[institute_access_list['display_name'] == institute]
institute_info

,base_uri,display_name,logo,support_contact
2,https://demo.eduvpn.nl/,Demo,https://static.eduvpn.nl/disco/img/demo.png,[mailto:eduvpn@surfnet.nl]


## or in case you select an organisation

In [77]:
organisation_info = organization_list[organization_list['display_name'] == organisation]
organisation_info

,display_name,org_id,secure_internet_home,keyword_list
561,SURFnet bv,https://idp.surfnet.nl,https://nl.eduvpn.org/,SURFnet bv SURF konijn powered by


In [51]:
info_base_url = institute_info['base_uri'].iloc[0]

In [52]:
info_url = info_base_url + 'info.json'
info = requests.get(info_url).json()['api']['http://eduvpn.org/api#2']

In [53]:
api_base_uri = info['api_base_uri']
token_endpoint = info['token_endpoint']
authorization_endpoint = info['authorization_endpoint']

In [54]:
port = get_open_port()
redirect_uri = f'http://127.0.0.1:{port}/callback'

In [55]:
oauth = OAuth2Session(client_id, redirect_uri=redirect_uri, auto_refresh_url=token_endpoint, scope=scope)

In [56]:
code_verifier = gen_code_verifier()
code_challenge = gen_code_challenge(code_verifier)
authorization_url, state = oauth.authorization_url(url=authorization_endpoint,
                                                   code_challenge_method=code_challenge_method,
                                                   code_challenge=code_challenge)

In [57]:
webbrowser.open(authorization_url)

True

In [58]:
response = one_request(port, lets_connect=False)

127.0.0.1 - - [04/May/2020 12:07:02] "GET /callback?code=eyJhbGciOiJFZERTQSIsInR5cCI6IkpXVCIsImtpZCI6IldMQTA5RWVfWHdoU0ZYalBOZ2ExT3lXRW40a1pUZ2xNR3ppR1RtRnM5MEEifQ.eyJ2Ijo1LCJ0eXBlIjoiYXV0aG9yaXphdGlvbl9jb2RlIiwiYXV0aF9rZXkiOiJmNzI5ZTlmZmZhZjFmMmIwYTg0MmQ4ODEzZDEyY2I4NyIsInVzZXJfaWQiOiJodHRwczpcL1wvc2EtZ3cuc3VyZmNvbmV4dC5ubFwvYXV0aGVudGljYXRpb25cL21ldGFkYXRhIWh0dHBzOlwvXC9kZW1vLmVkdXZwbi5ubFwvc2FtbCFlMTEzN2VmNzQyYWRmODMwOGFhYjdlMGE2YjY0M2M4ZTIyY2JmM2ZiIiwiY2xpZW50X2lkIjoib3JnLmVkdXZwbi5hcHAubGludXgiLCJzY29wZSI6ImNvbmZpZyIsInJlZGlyZWN0X3VyaSI6Imh0dHA6XC9cLzEyNy4wLjAuMTo1OTg2OVwvY2FsbGJhY2siLCJjb2RlX2NoYWxsZW5nZSI6IjEtSUJ1MzZNQUZEaHBTTGZKRlo0NnQtZUV0aGtDQmYwQTM2eExxeTBSVWMiLCJleHBpcmVzX2F0IjoiMjAyMC0wNS0wNFQxMDoxMjowMiswMDowMCJ9.8Mm3-GXbVhkbGoHrfaI2pSN3Xaux-M-5Z3lnorxv_JOQLl-KsAApTHY7S9rkzQqjdBznyZNRQII3wSo1UmJKDQ&state=5Wd5AYqOHagqeX27IKTgMbkNRdS9Vx HTTP/1.1" 200 -


In [59]:
code = response['code'][0]

In [60]:
assert(state == response['state'][0])

In [61]:
token = oauth.fetch_token(token_url=token_endpoint,
                          code=code,
                          code_verifier=code_verifier,
                          client_id=oauth.client_id,
                          include_client_id=True,
                          )

# profile list

In [62]:
profile_list_response = oauth.get(api_base_uri + '/profile_list')

In [63]:
profile_list_response.json()['profile_list']['data']

[{'profile_id': 'internet', 'display_name': 'internet', 'two_factor': False},
 {'profile_id': 'routes',
  'display_name': 'No rfc1918 routes',
  'two_factor': False}]

In [64]:
response = oauth.post(api_base_uri + '/create_keypair')
keypair = response.json()['create_keypair']['data']
private_key = keypair['private_key']
certificate = keypair['certificate']
common_name = common_name_from_cert(certificate.encode('ascii'))

# profile config

In [65]:
profile_id = 'internet'
response = oauth.get(api_base_uri + f'/profile_config?profile_id={profile_id}')
config = response.text

# check_certificate

In [66]:
response = oauth.get(api_base_uri + f'/check_certificate?common_name={common_name}')
assert(response.json()['check_certificate']['data']['is_valid'])

# system_messages

In [67]:
response = oauth.get(api_base_uri + '/system_messages')

In [68]:
response.json()['system_messages']['data']

[{'type': 'notification',
  'date_time': '2019-05-16T07:59:48Z',
  'message': 'Welcome to the eduVPN demo server by SURFnet. If you have any questions, please let us know by mailing us on eduvpn@surfnet.nl.'}]

# write networkmanager config

In [69]:
tmp = tempfile.NamedTemporaryFile(mode='w+t')
tmp.writelines(config)
tmp.seek(0)
filename = tmp.name

In [70]:
connection = None
for vpn_info in NM.VpnPluginInfo.list_load():
    print("TRY:  plugin %s" % (vpn_info.get_filename()))
    try:
        vpn_plugin = vpn_info.load_editor_plugin()
    except Exception as e:
        print("SKIP: cannot load plugin: %s" % (e))
        continue
    try:
        connection = vpn_plugin.import_(filename)
    except Exception as e:
        print("SKIP: failure to import %s" % (e))
        continue
    break

if connection is None:
    print("None of the VPN plugins was able to import \"%s\"" % (filename))
    sys.exit(1)

TRY:  plugin /usr/lib/NetworkManager/VPN/nm-openvpn-service.name


In [71]:
connection.normalize()

print("connection imported from \"%s\" using plugin \"%s\" (\"%s\", %s)" % (filename, vpn_info.get_filename(), connection.get_id(), connection.get_uuid()))

client = NM.Client.new(None)

main_loop = GLib.MainLoop()

def added_cb(client, result, data):
    try:
        client.add_connection_finish(result)
        print("The connection profile has been successfully added to NetworkManager.")
    except Exception as e:
        print("ERROR: failed to add connection: %s\n" % e)
    main_loop.quit()

client.add_connection_async(connection, True, None, added_cb, None)

main_loop.run()

connection imported from "/tmp/tmpcwd9cq6s" using plugin "/usr/lib/NetworkManager/VPN/nm-openvpn-service.name" ("tmpcwd9cq6s", 7713a7d5-fd15-4a1c-9251-a93663ba810f)
The connection profile has been successfully added to NetworkManager.
